In [ ]:
import multiprocess as mp
import os
import re
import time
from functools import partial
from itertools import product
from typing import Iterable, List, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm
from graph_state.graph_state import *

In [ ]:
def bell_sampling_diagonal_experiment_optimize(
    g: GraphState,
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    num_shots: Union[int, Iterable[int]],
    num_repeats: int,
    output_dir: str = "bell_diag_data",
    overwrite: bool = False
):
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [num_shots] if isinstance(num_shots, int) else list(num_shots)

    # 2. Create the directory for saving results if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # 3. Create all combinations of the parameters
    combinations = list(product(err_models, fidelities, shots_list))
    
    print(f"Starting Bell diagonal experiment for {g.n} qubits, {len(err_models)} error model(s), {len(fidelities)} fidelity value(s), and {len(shots_list)} shot setting(s).")
    # print(f"Starting experiment for {len(combinations)} parameter combinations...")
    # 4. Iterate through each combination, run the experiment, and save the result
    for err, fid, shots in tqdm.tqdm(combinations, desc="Running Experiments"):
        # Construct the output filename and filepath
        filename = f"bell_diag_{g.n}q_F{fid:.3f}_err_{err}_shots_{shots}.npy"
        filepath = os.path.join(output_dir, filename)
        # we need two copies at a time to run Bell sampling
        actual_shots = (shots + 1) // 2
        
        # Check if the file exists and skip if overwrite is False
        if not overwrite and os.path.exists(filepath):
            tqdm.tqdm.write(f"    Skipping, file already exists: {filepath}")
            continue
            
        all_diags_for_combo = []
        for _ in range(num_repeats):
            bell_samples = bell_sampling(g, err, fid, actual_shots)
            exps = expectation_value_of_observables_from_bell_bitpacked_parallelized(g, bell_samples)
            sqrt_exps_safe = np.sqrt(np.maximum(0, exps))
            diags = get_diagonals_from_all_stabilizer_observables(g, sqrt_exps_safe)
            all_diags_for_combo.append(diags)
            
        # Shape will be (num_repeats, number_of_diagonals)
        stacked_diags = np.array(all_diags_for_combo)
        np.save(filepath, stacked_diags)

In [2]:
def bell_sampling_diagonal_experiment(
    g: GraphState,
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    num_shots: Union[int, Iterable[int]],
    num_repeats: int,
    output_dir: str = "bell_diag_data",
    overwrite: bool = False
):
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [num_shots] if isinstance(num_shots, int) else list(num_shots)

    # 2. Create the directory for saving results if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # 3. Create all combinations of the parameters
    combinations = list(product(err_models, fidelities, shots_list))
    
    print(f"Starting Bell diagonal experiment for {g.n} qubits, {len(err_models)} error model(s), {len(fidelities)} fidelity value(s), and {len(shots_list)} shot setting(s).")
    # print(f"Starting experiment for {len(combinations)} parameter combinations...")
    # 4. Iterate through each combination, run the experiment, and save the result
    for err, fid, shots in tqdm.tqdm(combinations, desc="Running Experiments"):
        # Construct the output filename and filepath
        filename = f"bell_diag_{g.n}q_F{fid:.3f}_err_{err}_shots_{shots}.npy"
        filepath = os.path.join(output_dir, filename)
        # we need two copies at a time to run Bell sampling
        actual_shots = (shots + 1) // 2
        
        # Check if the file exists and skip if overwrite is False
        if not overwrite and os.path.exists(filepath):
            tqdm.tqdm.write(f"    Skipping, file already exists: {filepath}")
            continue
            
        all_diags_for_combo = []
        for _ in range(num_repeats):
            bell_samples = bell_sampling(g, err, fid, actual_shots)
            exps = expectation_value_of_observables_from_bell_bitpacked_parallelized(g, bell_samples)
            sqrt_exps_safe = np.sqrt(np.maximum(0, exps))
            diags = get_diagonals_from_all_stabilizer_observables(g, sqrt_exps_safe)
            all_diags_for_combo.append(diags)
            
        # Shape will be (num_repeats, number_of_diagonals)
        stacked_diags = np.array(all_diags_for_combo)
        np.save(filepath, stacked_diags)

def partial_tomo_experiment(
    g: GraphState,
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    total_shots: Union[int, Iterable[int]],
    num_repeats: int,
    output_dir: str = "tomo_data",
    overwrite: bool = False
):
    """
    Runs a partial tomography experiment for all combinations of the given parameters.

    This function accepts single values or iterables (lists, ranges) for the
    error model, fidelity, and shots. It then iterates through every possible
    combination, runs the simulation, and saves the resulting diagonal values
    of the density matrix to a .npy file.

    Args:
        g (GraphState): The graph state object.
        err_model (Union[str, List[str]]): A single error model string or a list of them.
        fidelity (Union[float, Iterable[float]]): A single fidelity value or an iterable.
        num_shots_per_obs (Union[int, Iterable[int]]): A single value for shots
            per observable, or an iterable of values.
        output_dir (str): The directory where the output .npy files will be saved.
        overwrite (bool): If False, skips the calculation if the output file
                          already exists. If True, it will always run and overwrite
                          any existing file.
    """
    # 1. Normalize all inputs to be lists so we can use itertools.product
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [total_shots] if isinstance(total_shots, int) else list(total_shots)

    # 2. Create the directory for saving results if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # 3. Create all combinations of the parameters
    combinations = list(product(err_models, fidelities, shots_list))
    
    N = 2 ** g.n
    

    print(f"Starting tomography experiment for {g.n} qubits, {len(err_models)} error model(s), {len(fidelities)} fidelity value(s), and {len(shots_list)} shot setting(s).")
    # print(f"Starting experiment for {len(combinations)} parameter combinations...")
    # 4. Iterate through each combination, run the experiment, and save the result
    for err, fid, budget_shots in tqdm.tqdm(combinations, desc="Running Experiments"):
        total_shots = (budget_shots // (N//2 + 2 ** (g.n - 2)) + 1) * N
        
        # Construct the output filename and filepath
        filename = f"tomo_{g.n}q_F{fid:.3f}_err_{err}_shots_{budget_shots}.npy"
        filepath = os.path.join(output_dir, filename)
        
        # Check if the file exists and skip if overwrite is False
        if not overwrite and os.path.exists(filepath):
            # Using a simple print inside tqdm can mess with the progress bar,
            # so we use tqdm.write for cleaner output.
            tqdm.tqdm.write(f"    Skipping, file already exists: {filepath}")
            continue
            
        # # Run the core simulation
        # exps = partial_tomo(g, err, fid, total_shots)
        # diags = get_diagonals_from_all_stabilizer_observables(g, exps)
        
        # # Save the numpy array
        # np.save(filepath, diags)
        # tqdm.tqdm.write(f"    Saved results to {filepath}")

        # Collect results from all repeats for this parameter combination
        all_diags_for_combo = []
        for _ in range(num_repeats):
            exps = partial_tomo(g, err, fid, total_shots)
            diags = get_diagonals_from_all_stabilizer_observables(g, exps)
            all_diags_for_combo.append(diags)
            
        # Stack the results into a single 2D NumPy array
        # Shape will be (num_repeats, number_of_diagonals)
        stacked_diags = np.array(all_diags_for_combo)

        # print(stacked_diags.shape)
        
        # Save the stacked numpy array
        np.save(filepath, stacked_diags)
        # tqdm.tqdm.write(f"    Saved ({stacked_diags.shape}) results to {filepath}")


In [3]:
def _run_bell_sampling_worker(combination: tuple, num_repeats: int, output_dir: str, overwrite: bool) -> str:
    """
    A single-process worker function for bell_sampling_fidelity_experiment.
    'combination' is a tuple: (g, err, fid, shots, stab_factor)
    
    Returns:
        A status string for logging.
    """
    # 1. Unpack the combination tuple
    g, err, fid, shots, stab_factor = combination

    # 2. Calculate numstab
    try:
        if stab_factor == '2n':
            numstab = g.n * 2
        elif stab_factor == 'n^2':
            numstab = g.n ** 2
        elif stab_factor.isdigit():
            numstab = int(stab_factor)
        else:
            raise RuntimeError(f"Stabilizer factor '{stab_factor}' not recognized")
    except Exception as e:
        return f"Failed (param error): {e}"

    # 3. Construct filename and check for overwrite
    filename = f"bell_fidelity_{g.n}q_F{fid:.3f}_err_{err}_shots_{shots}_numstab_{numstab}.npy"
    filepath = os.path.join(output_dir, filename)

    
    if not overwrite and os.path.exists(filepath):
        return f"Skipped (exists): {filename}"
    
    # 4. Run the actual experiment
    try:
        all_fidelities = []
        # Call bell_sampling once for all repeats, then split the data
        if (num_repeats * shots) <= 50_000_000:
            bell_samples = bell_sampling(g, err, fid, num_repeats * shots)
            samples_split = np.split(bell_samples, num_repeats)

            for samples in samples_split:
                est_fidelity = fidelity_estimation_via_random_sampling_bitpacked(g, numstab, samples)
                all_fidelities.append(est_fidelity)
        else:
            for _ in range(num_repeats):
                samples = bell_sampling(g, err, fid, shots)
                est_fidelity = fidelity_estimation_via_random_sampling_bitpacked(g, numstab, samples)
                all_fidelities.append(est_fidelity)
        
        # for samples in samples_split:
        #     est_fidelity = fidelity_estimation_via_random_sampling_bitpacked(g, numstab, samples)
        #     all_fidelities.append(est_fidelity)
            
        # 5. Save the result
        np.save(filepath, np.array(all_fidelities))
        return f"Saved ({len(all_fidelities)} repeats): {filename}"
    except Exception as e:
        return f"Failed (runtime error): {filename} with error: {e}"

def bell_sampling_fidelity_experiment(
    graphs: Union[GraphState, List[GraphState]],
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    num_shots: Union[int, Iterable[int]],
    num_repeats: int,
    stabilizer_factors: Union[str, List[str]],
    output_dir: str = "bell_fidelity_data",
    overwrite: bool = False
):
    """
    Runs a Bell sampling fidelity experiment in parallel for all combinations
    of parameters. Each combination is run in a separate process.
    """
    # 1. Normalize all inputs to be lists
    graphs_list = [graphs] if isinstance(graphs, GraphState) else list(graphs)
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [num_shots] if isinstance(num_shots, int) else list(num_shots)
    stabilizer_factors_list = [stabilizer_factors] if isinstance(stabilizer_factors, str) else list(stabilizer_factors)

    # 2. Create the directory for saving results
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # 3. Create all combinations of the parameters, including the graph
    combinations = list(product(graphs_list, err_models, fidelities, shots_list, stabilizer_factors_list))
    
    print(f"Starting Bell random sampling experiment for {len(graphs_list)} graph(s), "
          f"{len(err_models)} error model(s), {len(fidelities)} fidelity value(s), "
          f"{len(shots_list)} shot setting(s), and {len(stabilizer_factors_list)} stabilizer factor(s).")
    print(f"Total combinations (jobs) to run: {len(combinations)}")
    
    # 4. Set up the partial function for the worker
    worker_partial = partial(
        _run_bell_sampling_worker,
        num_repeats=num_repeats,
        output_dir=output_dir,
        overwrite=overwrite
    )
    
    # 5. Set up and run the multiprocessing pool
    num_processes = max(1, mp.cpu_count() - 2) # Leave some cores free
    print(f"Running on {num_processes} processes...")

    results = []
    with mp.Pool(processes=num_processes) as pool:
        # Use imap_unordered for efficiency, as order doesn't matter
        # Wrap the iterator with tqdm to get a progress bar
        for result in tqdm.tqdm(pool.imap_unordered(worker_partial, combinations), total=len(combinations), desc="Running Experiments"):
            results.append(result)
            # Optional: print results as they come in
            # tqdm.write(result) 

    print("\n--- Experiment Run Summary ---")
    saved_count = sum(1 for r in results if r.startswith("Saved"))
    skipped_count = sum(1 for r in results if r.startswith("Skipped"))
    failed_count = sum(1 for r in results if r.startswith("Failed"))
    print(f"Successfully saved: {saved_count}")
    print(f"Skipped (existed):  {skipped_count}")
    print(f"Failed:             {failed_count}")
    if failed_count > 0:
        print("\nFailures:")
        for r in results:
            if r.startswith("Failed"):
                print(f"  - {r}")

In [ ]:
"""
We generate the data for 8 qubit graph here.
    1. vary from F=0.5 to F=1 with N = 10^4
    2. vary from 10^2 to 10^5 with F = 0.9 (plotting later)
    X-axis will be log scale so we need to do log scaling X-axis (N)
"""

g = GraphState(8, "complete")
shots = 10_000
err_model = "depolarizing"
repeat = 1000
OVERWRITE = False

# first experiment
partial_tomo_experiment(
    g,
    err_model,
    fidelity=0.9,
    total_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    num_repeats=repeat,
    output_dir="even_more_tomo_data",
    overwrite=False,
)

# second experiment
partial_tomo_experiment(
    g,
    err_model,
    np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    shots,
    repeat,
    "even_more_tomo_data",
    False,
)

bell_sampling_diagonal_experiment(
    g,
    err_model,
    fidelity=0.9,
    num_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    num_repeats=repeat,
    output_dir="even_more_bell_diag_data",
    overwrite=False,
)

bell_sampling_diagonal_experiment(
    g,
    err_model,
    np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    shots,
    repeat,
    "even_more_bell_diag_data",
    False,
)

# # scalability experiment

# for n in range(2, 13):
#     g = GraphState(n, "complete")
#     partial_tomo_experiment(
#         g,
#         err_model,
#         0.9,
#         2 * 10_000,
#         25,
#         "tomo_data",
#         False,
#     )
#     bell_sampling_diagonal_experiment(
#         g,
#         err_model,
#         0.9,
#         20_000,
#         25,
#         "bell_diag_data",
#         False,
#     )

Saving experiment data to: 'even_more_tomo_data/'
Starting tomography experiment for 8 qubits, 1 error model(s), 1 fidelity value(s), and 50 shot setting(s).


Running Experiments: 100%|██████████| 50/50 [18:21<00:00, 22.03s/it]


Saving experiment data to: 'even_more_tomo_data/'
Starting tomography experiment for 8 qubits, 1 error model(s), 26 fidelity value(s), and 1 shot setting(s).


Running Experiments: 100%|██████████| 26/26 [09:07<00:00, 21.07s/it]


Saving experiment data to: 'even_more_bell_diag_data/'
Starting Bell diagonal experiment for 8 qubits, 1 error model(s), 1 fidelity value(s), and 50 shot setting(s).


Running Experiments:  44%|████▍     | 22/50 [5:41:31<9:56:34, 1278.39s/it]

In [4]:
graphs = [GraphState(i, 'complete') for i in range(21, 201)]
shots = 5_000
err_models = ['depolarizing', 'single-qubit-dephasing', 'bimodal']
# err_models = ['depolarizing']
stab_factors = '2n'
repeat = 50
OVERWRITE = False

# testing scalability of fidelity estimation
# bell_sampling_fidelity_experiment(graphs, err_models, 0.53, shots, repeat, stab_factors)

# testing varying stabs
num_qubits = 40
shots = 1_000_000
repeat = 50
stabilizer_counts = np.logspace(np.log10(num_qubits), np.log10(num_qubits**3), 50).astype(int)
stab_factors = list(map(str, stabilizer_counts))
bell_sampling_fidelity_experiment(GraphState(40), err_models, 0.53, shots, repeat, stab_factors)

# testing varying shots
# num_qubits = 40
# shots = np.logspace(2, 5, 50).astype(int)
# stab_factors = '2n'
# bell_sampling_fidelity_experiment(GraphState(40), err_models, 0.53, shots, repeat, stab_factors)

Saving experiment data to: 'bell_fidelity_data/'
Starting Bell random sampling experiment for 1 graph(s), 3 error model(s), 1 fidelity value(s), 1 shot setting(s), and 50 stabilizer factor(s).
Total combinations (jobs) to run: 150
Running on 12 processes...


Running Experiments: 100%|██████████| 150/150 [21:00:31<00:00, 504.21s/it]   


--- Experiment Run Summary ---
Successfully saved: 66
Skipped (existed):  84
Failed:             0
